Types of Transaction
--

In [ ]:
transaction_modes = [
    "UPI",               # Unified Payments Interface
    "INB",               # Internet Banking
    "IMP",               # Immediate Payment Service
    "NEFT",              # National Electronic Funds Transfer
    "RTGS",              # Real-Time Gross Settlement
    "Cheque",            # Paper Cheque
    "Cash Deposit",      # Cash Deposit to account
    "Cash Withdrawal",   # Withdrawal using cash
    "POS",               # Point of Sale (Card Payment)
    "DD",                # Demand Draft
    "SWIFT",             # Society for Worldwide Interbank Financial Telecommunication
    "Wire Transfer",     # Electronic bank transfer
    "ECS",               # Electronic Clearing Service
    "Bill Pay",          # Utility Bill Payments
    "M-wallet",          # Mobile Wallet Transfer
    "EMI",               # Equated Monthly Installment
    "EFT",               # Electronic Funds Transfer
    "ACH",               # Automated Clearing House
]


Dependencies
--

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)  # Display full content of each column
pd.set_option('display.max_columns', None)   # Display all columns
pd.set_option('display.width', 5000)         # Set display width


Pdf-Csv
--

In [3]:
import pdfplumber

def pdf_to_csv(pdf_path):
    pdf = pdfplumber.open(pdf_path)

    df = pd.DataFrame()

    for i in range(len(pdf.pages)):
        table = pdf.pages[i].extract_table()
        local_df = pd.DataFrame(table[1:], columns=table[0])
        df = pd.concat([df, local_df], ignore_index=True)
    return df

    


Merge ALL CSV'S
--

In [4]:
# df1=pd.read_csv("24-25.csv")
# df2=pd.read_csv("23-24.csv")
# df3=pd.read_csv("22-23.csv")

# df = pd.concat([df1, df2, df3], ignore_index=True)

# df.to_csv("Merge.csv",index=False)
# df.columns

PATHS
--

In [5]:
pdf_path=""
Processed_output_path=""

Segregate Columns with Mode/Name/ID/Note
--

In [7]:
import re
import pandas as pd

# List of all transaction modes
transaction_modes = [
    "UPI", "INB", "IMP", "NEFT", "RTGS", "Cheque", "Cash Deposit", "Cash Withdrawal", 
    "POS", "DD", "SWIFT", "Wire Transfer", "ECS", "Bill Pay", "M-wallet", "EMI", "EFT", "ACH"
]

pattern = re.compile(
    r"(?P<type>TO|BY) TRANSFER-(?P<method>[A-Z0-9]+)"
    r"/(?P<drcr>DR|CR)/(?P<id>\d+)/(?P<name>[^/]+)/(?P<bank>[A-Z]+)/(?P<upi_id>[^/]+)(?:/(?P<optional>[^-]+))?"
)

def extract_details(description):
    match = pattern.search(description)
    if match:
        return pd.Series({
            "Transaction Type": match.group("type"),
            "Transaction Mode": match.group("method"),
            "DR/CR Indicator": match.group("drcr"),
            "Transaction ID": match.group("id"),
            "Recipient Name": match.group("name"),
            "Bank": match.group("bank"),
            "UPI ID": match.group("upi_id"),
            "Note": match.group("optional") if match.group("optional") else "N/A"
        })
    else:
        # Handle unstructured descriptions by splitting on '/'
        split_data = description.split('/')
        
        # Extract Transaction Mode by checking against the list
        transaction_mode = "N/A"
        for mode in transaction_modes:
            if mode in description:
                transaction_mode = mode
                break

        return pd.Series({
            "Transaction Type": "BY" if "BY" in description else "TO",
            "Transaction Mode": transaction_mode,
            "DR/CR Indicator": "N/A",
            "Transaction ID": split_data[1] if len(split_data) > 1 else 'N/A',
            "Recipient Name": split_data[2] if len(split_data) > 2 else 'N/A',
            "Bank": "N/A",
            "UPI ID": "N/A",
            "Note": split_data[3] if len(split_data) > 3 else 'N/A'
        })

text = "BY TRANSFER-INBIMPS409519772968/9890160567/XX8237/Son-"

# print(extract_details(text))


Csv-Organized
--

In [8]:
df=pdf_to_csv(pdf_path)

df = df.replace('\n', '', regex=True)

df = df.rename(columns={
    'Txn Date': 'Transaction Date',
    'Value\nDate': 'Value Date',
    'Description': 'Description',
    'Ref No./Cheque\nNo.': 'Reference No./Cheque No.',
    'Debit': 'Debit',
    'Credit': 'Credit',
    'Balance': 'Balance'
})
df=df.drop(columns=['Value Date'])
df_extracted = df['Description'].apply(extract_details)

df = pd.concat([df, df_extracted], axis=1)

# df.to_csv(csv_path, index=False)


Converting Dtypes
--

In [7]:
df['Transaction Date'] = df['Transaction Date'].str.replace(r'(\d{1,2})\s*([A-Za-z]{3})(\d{4})', r'\1 \2 \3', regex=True)
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format='%d %b %Y', dayfirst=True)

df['Debit'] = pd.to_numeric(df['Debit'].replace({',': '', 'nan': None, ' ': ''}, regex=True), errors='coerce')
df['Credit'] = pd.to_numeric(df['Credit'].replace({',': '', 'nan': None, ' ': ''}, regex=True), errors='coerce')
df['Balance'] = pd.to_numeric(df['Balance'].replace({',': '', 'nan': None, ' ': ''}, regex=True), errors='coerce')

categorical_cols = ['Transaction_Type', 'Transaction_Mode', 'DR/CR Indicator', 
                    'Recipient_Name', 'Bank', 'Upi_Id', 'Note']

for col in categorical_cols:
    df[col] = df[col].astype('category')


df['Reference No./Cheque No.'] = df['Reference No./Cheque No.'].astype(str)
df['Transaction ID'] = df['Transaction ID'].astype(str)


Handling NAN
--

In [11]:
df['Debit'] = pd.to_numeric(df['Debit'], errors='coerce').fillna(0)
df['Credit'] = pd.to_numeric(df['Credit'], errors='coerce').fillna(0)

def determine_dr_cr(df):
    for i in range(1, len(df)):  
        
        balance_diff = df.loc[i, 'Balance'] - df.loc[i-1, 'Balance']
        
        if balance_diff > 0:
            df.loc[i, 'DR/CR Indicator'] = 'CR'
        else:
            df.loc[i, 'DR/CR Indicator'] = 'DR'
    return df

def update_transaction_mode(row):
    if 'ATM' in row['Description']:
        row['Transaction Mode'] = 'ATM'
    return row

df = df.apply(update_transaction_mode, axis=1)

df = determine_dr_cr(df)


In [23]:

# df=pd.read_csv("Merge_Proccessed.csv")
# df.columns


Features
--

In [18]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

def feature_engineering(df):
    # Ensure 'Transaction Date' is datetime
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

    # Categorical day of the week
    cat_type = CategoricalDtype(categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
    
    # Extract time-based features
    df['dayofweek'] = df['Transaction Date'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['weekday'] = df['Transaction Date'].dt.day_name().astype(cat_type)
    df['quarter'] = df['Transaction Date'].dt.quarter
    df['month'] = df['Transaction Date'].dt.month
    df['year'] = df['Transaction Date'].dt.year
    df['dayofyear'] = df['Transaction Date'].dt.dayofyear
    df['dayofmonth'] = df['Transaction Date'].dt.day
    df['weekofyear'] = df['Transaction Date'].dt.isocalendar().week
    df['weekofmonth'] = df['Transaction Date'].apply(lambda x: (x.day - 1) // 7 + 1)
    
    df['date_offset'] = (df['Transaction Date'].dt.month * 100 + df['Transaction Date'].dt.day - 320) % 1300
    df['is_weekend'] = df['dayofweek'].apply(lambda x: 1 if x >= 5 else 0)


    # Start/End of period flags
    df['is_month_start'] = df['Transaction Date'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['Transaction Date'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['Transaction Date'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['Transaction Date'].dt.is_quarter_end.astype(int)
    
    # Create transaction amount column
    df['transaction_amount'] = df['Debit'].fillna(df['Credit'])
    df['is_large_transaction'] = df['transaction_amount'].apply(lambda x: 1 if x > 1000 else 0)

    # Transaction count per day, week, and month
    df['transaction_count_per_day'] = df.groupby(df['Transaction Date'].dt.date)['Transaction ID'].transform('count')
    df['transaction_count_per_week'] = df.groupby(df['weekofyear'])['Transaction ID'].transform('count')
    df['transaction_count_per_month'] = df.groupby(df['month'])['Transaction ID'].transform('count')

    # Average transaction per day
    df['average_transaction_per_day'] = df.groupby(df['Transaction Date'].dt.date)['transaction_amount'].transform('mean')

    # Cumulative spent per day
    df['cumulative_spent_per_day'] = df.groupby(df['Transaction Date'].dt.date)['Debit'].cumsum()

    # Time since last transaction
    df = df.sort_values(by=['Transaction Date'])
    df['time_since_last_transaction'] = df['Transaction Date'].diff().dt.total_seconds() / 3600  # in hours
    df['time_since_last_transaction'].fillna(0, inplace=True)

    # Rolling average transactions over past 7 & 30 days
    df['rolling_avg_transaction_7d'] = df['transaction_amount'].rolling(window=7, min_periods=1).mean()
    df['rolling_avg_transaction_30d'] = df['transaction_amount'].rolling(window=30, min_periods=1).mean()

    return df

# Example Usage
df = feature_engineering(df)


C:\Users\yashs\AppData\Local\Temp\ipykernel_17600\4200821036.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['time_since_last_transaction'].fillna(0, inplace=True)


In [21]:
import dtale

dtale.show(df)

Save Processed CSV
--

In [24]:
df.to_csv(Processed_output_path, index=False)


In [25]:
df.columns

Index(['Transaction Date', 'Value Date', 'Description', 'Reference No./Cheque No.', 'Debit', 'Credit', 'Balance', 'Transaction Type', 'Transaction Mode', 'DR/CR Indicator', 'Transaction ID', 'Recipient Name', 'Bank', 'UPI ID', 'Note', 'dayofweek', 'weekday', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'date_offset', 'is_weekend', 'weekofmonth', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'hour', 'minute', 'second', 'transaction_amount', 'is_large_transaction', 'transaction_count_per_day', 'transaction_count_per_week', 'transaction_count_per_month', 'average_transaction_per_day', 'cumulative_spent_per_day', 'time_since_last_transaction', 'rolling_avg_transaction_7d', 'rolling_avg_transaction_30d'], dtype='object')

Print N/A
--

In [26]:
na_count_per_column = df.applymap(lambda x: x == "N/A").sum()

# Print the result
print("Number of 'N/A' values per column:")
# print(na_count_per_column)

Number of 'N/A' values per column:


C:\Users\yashs\AppData\Local\Temp\ipykernel_17600\703361585.py:1: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



Print Everthing
--

In [11]:
def print_transaction_details(df):
    """
    Prints every row with all the extracted details from the DataFrame.
    """
    for index, row in df.iterrows():
        print(f"Row {index}:")
        for col in df.columns:
            print(f"  {col}: {row[col]}")
        print("-" * 50)  # Separator for better readability
# print_transaction_details(df)
        

Holidays
--

In [ ]:
import holidays
import pandas as pd

indian_holidays = holidays.India(years=2024)

df = pd.DataFrame(indian_holidays.items(), columns=["Date", "Holiday"])
df["Date"] = pd.to_datetime(df["Date"])

df
